In [2]:
import pandas as pd
import datetime as dt 
import time as t
import numpy as np
import openpyxl
from  matplotlib import pyplot as plt

In [5]:
df5 = pd.read_pickle('df4.pkl')

In [6]:
df5

,InvoiceNo,StockCode,Description,Quantity,Backorder_Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYear,InvoiceMonth,TotalSales
0,536365,85123A,White hanging heart t-light holder,6,0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,December,15.30
1,536365,71053,White metal lantern,6,0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,December,20.34
2,536365,84406B,Cream cupid hearts coat hanger,8,0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010,December,22.00
3,536365,84029G,Knitted union flag hot water bottle,6,0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,December,20.34
4,536365,84029E,Red woolly hottie white heart.,6,0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,December,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,Pack of 20 spaceboy napkins,12,0,2011-12-09 12:50:00,0.85,12680.0,France,2011,December,10.20
541905,581587,22899,Children's apron dolly girl,6,0,2011-12-09 12:50:00,2.10,12680.0,France,2011,December,12.60
541906,581587,23254,Childrens cutlery dolly girl,4,0,2011-12-09 12:50:00,4.15,12680.0,France,2011,December,16.60
541907,581587,23255,Childrens cutlery circus parade,4,0,2011-12-09 12:50:00,4.15,12680.0,France,2011,December,16.60


In [27]:
rfm = df5.groupby('CustomerID').agg({
    "InvoiceDate" : lambda x : (pd.Timestamp.today().normalize() - pd.to_datetime(x.max()) ).days,
    "StockCode" : "count",
    "TotalSales" : "sum",
    "Country" : "count"
})

In [30]:
rfm.columns = ['recency','total_stock','sales_each','how_many_country']

In [34]:
rfm['r_score'] = pd.qcut(rfm['recency'],5,[5,4,3,2,1])
rfm['f_score'] = pd.qcut(rfm['total_stock'],5,[5,4,3,2,1])
rfm['m_score'] = pd.qcut(rfm['sales_each'],5,[5,4,3,2,1])

In [54]:
def segment(x):
    if x <= 5:
        return "Low Level Customer"
    elif x > 5 and x <= 10 :
        return "Valuable customer"
    elif x > 10:
        return "Best Customer"

rfm['total_score'] =  rfm['r_score'].astype(int) + rfm['f_score'].astype(int) + rfm['m_score'].astype(int)       

In [55]:
rfm['customer_type'] = rfm['total_score'].apply(segment)

In [56]:
rfm

,recency,total_stock,sales_each,how_many_country,r_score,f_score,m_score,total_score,customer_type
CustomerID,,,,,,,,,
12346.0,5415,2,77183.60,2,1,5,1,7,Valuable customer
12347.0,5092,182,4310.00,182,5,1,1,7,Valuable customer
12348.0,5165,31,1797.24,31,2,3,2,7,Valuable customer
12349.0,5108,73,1757.55,73,4,2,2,8,Valuable customer
12350.0,5400,17,334.40,17,1,4,4,9,Valuable customer
...,...,...,...,...,...,...,...,...,...
18281.0,5270,7,80.82,7,1,5,5,11,Best Customer
18282.0,5097,13,178.05,13,5,5,5,15,Best Customer
18283.0,5093,756,2094.88,756,5,1,1,7,Valuable customer


In [58]:
rfm['recency'].min() ##5090 
rfm['recency'].max() ## 5463

np.int64(5463)

In [59]:
def is_churned_customer(x):
     threshold = 5200
     if x >= threshold :
         return "Not a Churned Customer"
     else:
         return "Churned Customer"

rfm['Churned_Cat'] = rfm['recency'].apply(is_churned_customer)         



In [60]:
rfm

,recency,total_stock,sales_each,how_many_country,r_score,f_score,m_score,total_score,customer_type,Churned_Cat
CustomerID,,,,,,,,,,
12346.0,5415,2,77183.60,2,1,5,1,7,Valuable customer,Not a Churned Customer
12347.0,5092,182,4310.00,182,5,1,1,7,Valuable customer,Churned Customer
12348.0,5165,31,1797.24,31,2,3,2,7,Valuable customer,Churned Customer
12349.0,5108,73,1757.55,73,4,2,2,8,Valuable customer,Churned Customer
12350.0,5400,17,334.40,17,1,4,4,9,Valuable customer,Not a Churned Customer
...,...,...,...,...,...,...,...,...,...,...
18281.0,5270,7,80.82,7,1,5,5,11,Best Customer,Not a Churned Customer
18282.0,5097,13,178.05,13,5,5,5,15,Best Customer,Churned Customer
18283.0,5093,756,2094.88,756,5,1,1,7,Valuable customer,Churned Customer
